In [93]:
from pyzotero import zotero
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
from contextlib import closing
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import requests
import io
import PyPDF2 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import sys
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from StringIO import StringIO
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter

ModuleNotFoundError: No module named 'pdfminer'

In [5]:
def get_zotero_lib(library_id, library_type, api_key, n_articles):
    zot = zotero.Zotero(library_id, library_type, api_key)
    items = zot.top(limit=n_articles)
    return items

def scrape_item(item):
    url = item['data']['url']
    urllib_obj = urlopen(url)
    html = BeautifulSoup(urllib_obj.read());
    tags = {re.compile('h\d') : True, 
            'p' : True, 
            'blockquote': True}
    return '\n'.join([p.get_text() for p in html.find_all(tags)])

In [43]:
library_id = 2269135
library_type = 'group'
api_key = 'bPraFnu08tmj8mtlAyuKnnNY'
n_articles = 30
items = get_zotero_lib(library_id, library_type, api_key, n_articles)

In [44]:
texts = []
for idx, item in enumerate(items):
    try:
        text = scrape_item(item)
        item['data']['article_body'] = text
    except HTTPError as e:
        print(e)
        print(item['data']['url'])
        item['data']['article_body'] = None

HTTP Error 410: Gone
https://www.nytimes.com/reuters/2018/07/13/business/13reuters-tech-rights.html


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1051)>

In [ ]:
items

## Try to download pdfs

In [165]:
# from http://survivalengineer.blogspot.com/2014/04/parsing-pdfs-in-python.html
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
import io
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
class MyParser(object):
    def __init__(self, pdf):
        ## Snipped adapted from Yusuke Shinyamas 
        #PDFMiner documentation
        # Create the document model from the file
        parser = PDFParser(pdf)
        document = PDFDocument(parser)
        # Try to parse the document
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        # Create a PDF resource manager object 
        # that stores shared resources.
        rsrcmgr = PDFResourceManager()
        # Create a buffer for the parsed text
        retstr = io.StringIO()
        # Spacing parameters for parsing
        laparams = LAParams()
        codec = 'utf-8'
 
        # Create a PDF device object
        device = TextConverter(rsrcmgr, retstr, 
                               codec = codec, 
                               laparams = laparams)
        # Create a PDF interpreter object
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        # Process each page contained in the document.
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
         
        self.records            = []
         
        lines = retstr.getvalue().splitlines()
        for line in lines:
            self.handle_line(line)
     
    def handle_line(self, line):
        # Customize your line-by-line parser here
        self.records.append(line)

In [166]:
# define keys
library_id = 2269135
library_type = 'group'
api_key = 'bPraFnu08tmj8mtlAyuKnnNY'
collection_key = 'VFDISWVI' # key for api_testing collection

#read items
zot = zotero.Zotero(library_id, library_type, api_key)
items = zot.collection_items(collection_key)

In [172]:
for item in items:
    if item['data']['itemType'] != "attachment":
        if item['meta']['numChildren'] > 0:
            attachment = zot.children(item['data']['key'])
            if len(attachment[0]['data']['url']) > 0:
                pdf_url = attachment[0]['data']['url']
                r = requests.get(pdf_url)
                f = io.BytesIO(r.content)
            else:
                zot.dump(attachment[0]['key'], path ='./pdfs')
                f = open('./pdfs/'+attachment[0]["data"]["filename"], 'rb')
            p = MyParser(f)
            print('\n'.join(p.records))

23 Statistics That Prove Men
Dominate The Tech World

Megan Rose Dickey

The lack of women in technology
conversation is nothing new. 

Over the last few months, the topic has
been fueled by events like "Donglegate"
and its backlash.

It's since become common knowledge
that women are underrepresented in tech
jobs, but what about their representation
in the tech world as a whole?

Getty Images/Kevork Djansezian

Here are statistics that shed some light on both:

In the U.S., women are actually more likely to use social networking sites. In
December 2012, 71% of women compared to 62% of men surveyed were using
social networking sites, according to Pew Internet.
Google, LinkedIn, and Reddit dominated by
men

Here's the breakdown of the actual user base make-up, according to a 2012
report by marketing firm DigitalFlashNYC. Note: Women make up the
majority of users on Twitter, Facebook, Zynga, and Pinterest.

Stat 1: LinkedIn: 63% men and 37% women

Stat 2: Google+: 71% men and 29% women


